In [ ]:
#%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

In [ ]:
# from fastai.column_data import *
# from fastai.conv_learner import *
# from fastai.dataset import *
from fastai.torch_imports import save_model, load_model

In [ ]:
from models import *
from utils import *

# Experiment Acyn

In [ ]:
EXPCODE = "oldfasion"

In [ ]:
ls -f KITTI/ | grep .csv

In [ ]:
train, val = pd.read_csv('./KITTI/training_192_640_pre.csv'), pd.read_csv('./KITTI/validation_192_640_pre.csv')
train, val = shuffle(train), shuffle(val);

In [ ]:
train.head()

In [ ]:
val.head()

In [ ]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [ ]:
trn_camera.head()

In [ ]:
arch = f
bs = 8
PATH = 'Fastai_TRN'
DPATH = "KITTI/"
verbose = 100

In [ ]:
f

In [ ]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [ ]:
stats = imagenet_stats
tfm_norm = Normalize(*stats, tfm_y=TfmType.NO) 
tfm_denorm = Denormalize(*stats)

In [ ]:
trn_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)
val_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)

In [ ]:
tfms = (trn_tfms, val_tfms)

MD1 = get_MD(trn0, val0, tfms, bs, DPATH, PATH)
MD2 = get_MD(trn1, val1, tfms, bs, DPATH, PATH)
MD3 = get_MD(trn2, val2, tfms, bs, DPATH, PATH)
MDcam = get_cam(trn_camera, val_camera, bs)

In [ ]:
imgnet_mean, imgnet_std = torch.from_numpy(stats[0]).float(), torch.from_numpy(stats[1]).float()
imgnet_mean, imgnet_std = imgnet_mean.view(1,3,1,1), imgnet_std.view(1,3,1,1)
denorm = denormer(imgnet_mean, imgnet_std)

In [ ]:
def train(m, opt, MD1, MD2, MD3, MDcam, denorm):
    collector = LossCollect(6)
    
    m.train()
    DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
    losses = []  
    
    #for i in range(2): # just for testing
    for i in range(len(MD1.trn_ds)//bs-len(losses)):
        t1 = time.time()
        opt.zero_grad()
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])
        
        d1, d2, d3, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1), denorm(imgs2), denorm(imgs3) 
        #pdb.set_trace()
        loss, details = l(d1, d2, d3, trans, rotation, imgs1, imgs2, imgs3, cam)
        loss.backward()
        opt.step()
        t2 = time.time()
        
        collector.collect(
            loss.data[0],
            details[0].data[0],
            details[1].data[0],
            details[2].data[0],
            details[3].data[0],
            t2-t1
            )
        
        # total appr smooth ssim l1
        if i%verbose == 0: print("{:.4f}, {:.4f}, {:.4E}, {:.4f}, {:.4f}, {:.2f}s/batch".format(*collector.show()))
    return collector.collections[0]  #return the total losses

In [ ]:
def evaluate(m, MD1, MD2, MD3, MDcam, folder):
    
    DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
    m.eval()
    for i in range(3): 
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
        d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1, volatile=True), denorm(imgs2, volatile=True), denorm(imgs3, volatile=True) 
        #pdb.set_trace()
        d2 = d2s[0]
        # d2 = F.upsample(input=d2, scale_factor=2**di, mode='bilinear')
        cx12, cy12, dm12 = l.appr.offset.forward(trans=trans[:,0], rotation=rotation[:,0], inv_depth = d2, camera = cam)
        #pdb.set_trace()

        x12, ivm12 = l.appr.sampler.forward(imgs1, cx12, cy12)
        ivm12.volatile = True

        del dm12 
        del ivm12
        
        imgs2 = tonp(imgs2.permute(0,2,3,1))
        x12 = tonp(x12.permute(0,2,3,1))
        d2 = tonp(d2)
        
        for j in range(bs):
            save_res(imgs2[j], x12[j], d2[j][0], folder/"{}{}.png".format(i,j))

In [ ]:
def fit(epoch, m, opt, MD1, MD2, MD3, MDcam):
    for i in range(epoch):
        print("--------------------epoch {} start:----------------------".format(i))
        losses = train(m, opt, MD1, MD2, MD3, MDcam)
        losses= [ str(loss) for loss in losses ]
        folder = Path("./tmp")
        folder.mkdir(exist_ok=True)
        file = folder / 'epoch{}.log'.format(i)
        with file.open('w') as f:
            f.write("\n".join(losses))
        save_model(m, str(folder / "epoch{}.M".format(i)))

In [ ]:
expfolder = Path('./experiment/') / EXPCODE
expfolder.mkdir(parents=True, exist_ok=True)

In [ ]:
# this one use mean pixel loss with old fasion in view mask to filter out the unavalible pixels.
m = TriDepth(get_base(f, cut), 1, setting=[False, False, False], is_per_pixel_min=False, is_mask=True).cuda()
l = Loss(smooth_scale=0.001, appr_scale=0.85, warp_setting=[True, True, True, True]).cuda()
opt = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr =1e-4)


In [ ]:
epoch = 8
#     set_trainable(m.depth.rn, False)
for i in range(epoch):    
    thisfolder = expfolder/"epoch_{}".format(i+1)
    thisfolder.mkdir(exist_ok=True)

    print("------------------training model-----------------")
    losses = train(m, opt, MD1, MD2, MD3, MDcam, denorm)
    print("------------------saving model-------------------")
    save_model(m, thisfolder/"model.M")
    print("------------------writting log-------------------")
    logger = thisfolder/"log"
    losses = [ str(loss) for loss in losses]
    with logger.open('w') as f: f.write("\n".join(losses))
    print("---------------------predicting------------------")    
    predictfolder = thisfolder/'predicts'
    predictfolder.mkdir(exist_ok=True)
    evaluate(m, MD1, MD2, MD3, MDcam, predictfolder)

In [ ]:
DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
m.eval()


In [ ]:
imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        

In [ ]:
d2s[0].min(), d2s[0].max()